<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%run "C:\Users\BILHA\Google Drive\xRepos\xUtilz\envz_only_loader.ipynb"

In [2]:
## Functions 


"""
get a plot object with pretty set for matplotlib  
"""
def plotObject(x=12, y=7, nr=1, nc=1):
    fig, ax = plt.subplots( nrows=nr, ncols=nc, figsize=(x,y))
    return fig, ax

"""
Define n for different group levels 
    - Too Slow or never terminates TODO
"""
def sampleStructure(dtset, colz=None, grp=None, rnd=1, fx=None ):
    fx  =  fx if fx else [np.mean, sum ] # np.unique,
    if colz and grp: return dtset[colz+[grp]].groupby(grp).agg( fx ).round(rnd) 
    elif colz: return dtset[colz].agg( fx ).round(rnd)
    elif grp: return dtset.groupby(grp).agg(fx ).round(rnd)
    else: return dtset.agg( fx ).round(rnd)

"""
Don't know why needed this one instead of using default describe 
"""
def zDescribe( dtset, colz=None, rnd=2):
    colz = colz if colz else dtset.columns
    
    return pd.DataFrame( { 
        "Sum": og_only_dataset[ colz  ].sum().round(rnd), 
        "Avg": og_only_dataset[ colz ].mean().round(rnd),
        "Min": og_only_dataset[ colz  ].min().round(rnd),
        "Max" : og_only_dataset[ colz  ].max().round(rnd),
        "SD" : og_only_dataset[ colz  ].max().round(rnd),
              }).T

"""
Using default describe - TODO pretty format
"""
def zSummarize( dtset, colz=None, rnd=2):
    colz = colz if colz else dtset.columns 
    
    return (
        dtset[ colz ].describe( ).round( rnd ),
        dtset[ colz ].describe( exclude=[ np.number] )
    )

"""
Find a word = check if a cell has one of a given list of words and return the key of the list it has first
"""
def searchString( word, inDict): 
    for k, v in inDict.items():
        v = pd.Series( v ).apply( lambda s: s.upper() )
        #print( "%s [%s]"%(k,v ) )
        rgx = re.compile( r'\b(?:%s)\b' % '|'.join( v ) ) 
        if pd.isnull( word ):
            return "No Response"
        
        if rgx.search( word.upper() ) :
            return k 

    
## TODO: with margins + fq% 
"""
Creates buckets based on percentiles
"""
def zPercentiles(dtset, colz=None, fxz=None, incdz=None ): 
    pct = [.2, .4, .5, .6, .8]
    incdz='all'
    if( colz): return dtset[colz].describe( percentiles=pct, include=incdz).round(2)
    else: return dtset.describe(percentiles=pct, include=incdz).round(2)


## TODO: update to sns and with group by
"""
blot distributions for a given list of fields
"""
def plotDistz(dtset, colz=None, grp=None, stk=False):
    fig, ax = plotObject()
    
    if colz and grp: return dtset.groupby(grp).hist(colz, stacked=stk,ax=ax)
    elif colz: return dtset.hist(colz, stacked=stk,ax=ax)
    elif grp: return dtset.groupby(grp).hist(stacked=stk,ax=ax)
    else: return dtset.hist(stacked=stk,ax=ax)

    #if( grpz ) return dtset.plot( colz, kind="hist", legend=True, alpha=0.4, ax=ax ) 
    #return dtset.hist(colz,stacked=True,ax=ax)


def plotBoxz(dtset, colz=None, grp=None):
    fig, ax = plotObject()
    
    if colz and grp: return dtset.boxplot( colz, grp, ax=ax) 
    elif colz: return dtset.boxplot( colz, ax=ax)
    elif grp: return dtset.boxplot( grp, ax=ax)
    else: return dtset.boxplot( ax=ax)

def fqTablez( dtset, grp, colz=None, margz=True, pct=False):
    colz = colz if colz else dtset.columns
    
    return pd.Series( colz ).apply( lambda x: pd.crosstab( dtset[x], grp, margins=margz) ) 
    #pd.crosstab( dataset.Females, dataset["Age Group"], margins=True).apply( lambda x: x/len(dataset)).round(4)*100

    
def frequenciez(dtset, colz=None, grp=None, flipaxis=False, margz=True):
    #if( grp):  grp = pd.Grouper(key=grp, sort=True)
    # 1.tabulate 
    # pd.crosstab( dtset[colz], dtset[grp], margins=margz)
    
    # 2.barplot  -- factorplot  
    if( flipaxis):
        if(colz and grp): return pd.Series(colz).apply(lambda x: sns.countplot(y=x, hue=grp, data=dtset) )
        elif(colz): return pd.Series(colz).apply(lambda x: sns.countplot( y=x, data=dtset) )
        elif( grp ): 
            colz = dtset.columns() 
            return pd.Series(colz).apply(lambda x: sns.countplot( y=x, hue=grp, data=dtset) )
        else: 
            colz = dtset.columns() 
            return pd.Series(colz).apply(lambda x: sns.countplot( y=x, data=dtset) )
    
    else:
        if(colz and grp): return pd.Series(colz).apply(lambda x: sns.countplot(x=x, hue=grp, data=dtset) )
        elif(colz): return pd.Series(colz).apply(lambda x: sns.countplot( x=x, data=dtset) )
        elif( grp ): 
            colz = dtset.columns() 
            return pd.Series(colz).apply(lambda x: sns.countplot( x=x, hue=grp, data=dtset) )
        else: 
            colz = dtset.columns() 
            return pd.Series(colz).apply(lambda x: sns.countplot( x=x, data=dtset) )

    

def corrz(dtset, colz, mthd="pearson"):
    return dtset[colz].corr( method=mthd).round(3)


def plotCorrz( cor_outz ):
    # Generate a mask for the upper triangle
    mask = np.zeros_like(cor_outz, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(7, 7))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    #TODO: pretty print tables : cor_outz, 
    return (cor_outz, sns.heatmap(cor_outz, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}) )

## TODO: generalize to smalls 
def quintilize(dtset, col, nbGrpz=5, labelzPrefix="Quint"):
    labelz = []
    for i in range(nbGrpz):
        labelz.append( labelzPrefix+"_"+str(i+1))
    dtset["Quints_"+col] = pd.qcut( dtset[col], nbGrpz, labels=labelz, duplicates='drop') 
    

def checkMissing(dtset):
    #dataset.fillna( np.nan )
    return "\nShape # records, # variables", dtset.shape, "\n\n\n Counts of Missing Values \n", dtset.apply( lambda x: sum(x.isnull() ), axis=0)

#def anonymizeSimple():
   





'\nget a plot object with pretty set for matplotlib  \n'

'\nDefine n for different group levels \n    - Too Slow or never terminates TODO\n'

"\nDon't know why needed this one instead of using default describe \n"

'\nUsing default describe - TODO pretty format\n'

'\n\n'

'\nCreates buckets based on percentiles\n'

In [ ]:
### Dash 
ext_css = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

def show_app( app, port=9999, width=700, height=350, offline=False, in_binder=None):
    url = "http://localhost:%d"%port 
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, width=width, height=height) 
    disp2.display_html( iframe, raw=True)
    return app.run_server(debug=False, host='0.0.0.0', port=port)

